# TMDB Box Office Prediction

The notebook's purpose is to produce submissions for the [Kaggle Competition](https://www.kaggle.com/c/tmdb-box-office-prediction/data).
The challenge is to predict the box office of a movie, given a number of factors.

In [3]:
# data loading and neccessary libraries import
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [4]:
PATH = './datasets/tmdb-box-office-prediction/'
train_location = os.path.join(PATH, 'train.csv')
test_location = os.path.join(PATH, 'test.csv')

In [6]:
train = pd.read_csv(train_location)
test = pd.read_csv(test_location)

In [7]:
train.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,...,2/20/15,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,...,8/6/04,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435
2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,...,10/10/14,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000
3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,...,3/9/12,122.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000
4,5,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,...,2/5/09,118.0,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",Released,NaN,Marine Boy,NaN,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",3923970


In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 23 columns):
id                       3000 non-null int64
belongs_to_collection    604 non-null object
budget                   3000 non-null int64
genres                   2993 non-null object
homepage                 946 non-null object
imdb_id                  3000 non-null object
original_language        3000 non-null object
original_title           3000 non-null object
overview                 2992 non-null object
popularity               3000 non-null float64
poster_path              2999 non-null object
production_companies     2844 non-null object
production_countries     2945 non-null object
release_date             3000 non-null object
runtime                  2998 non-null float64
spoken_languages         2980 non-null object
status                   3000 non-null object
tagline                  2403 non-null object
title                    3000 non-null object
Keywords             

## -- Plan of Attack
1. Clean the dataset, which includes a train and the test set.
2. Figure out which features correlate with the revenue.
3. Derive new features - this can occur at other times as well.
4. Evaluate and choose the best performing model.
5. Optimize the chosen model.

## 1. Cleaning Missing values and EDA


In [24]:
train.shape, test.shape

((3000, 23), (4398, 22))

In [20]:
def get_missing_by_columns(df):
    '''
        Prints the columns with missing data and what percentage are they.
    '''
    for column in list(df.columns):
        missing = df[df[column].isnull()].shape[0]
        if missing > 0:
            missing_perc = missing / df.shape[0] * 100
            print(f'{column} has {np.round(missing_perc, 2)} % missing. Count missing: {missing}')

In [22]:
get_missing_by_columns(train)

belongs_to_collection has 79.87 % missing. Count missing: 2396
genres has 0.23 % missing. Count missing: 7
homepage has 68.47 % missing. Count missing: 2054
overview has 0.27 % missing. Count missing: 8
poster_path has 0.03 % missing. Count missing: 1
production_companies has 5.2 % missing. Count missing: 156
production_countries has 1.83 % missing. Count missing: 55
runtime has 0.07 % missing. Count missing: 2
spoken_languages has 0.67 % missing. Count missing: 20
tagline has 19.9 % missing. Count missing: 597
Keywords has 9.2 % missing. Count missing: 276
cast has 0.43 % missing. Count missing: 13
crew has 0.53 % missing. Count missing: 16


In [23]:
get_missing_by_columns(test)

belongs_to_collection has 80.06 % missing. Count missing: 3521
genres has 0.36 % missing. Count missing: 16
homepage has 67.71 % missing. Count missing: 2978
overview has 0.32 % missing. Count missing: 14
poster_path has 0.02 % missing. Count missing: 1
production_companies has 5.87 % missing. Count missing: 258
production_countries has 2.32 % missing. Count missing: 102
release_date has 0.02 % missing. Count missing: 1
runtime has 0.09 % missing. Count missing: 4
spoken_languages has 0.95 % missing. Count missing: 42
status has 0.05 % missing. Count missing: 2
tagline has 19.62 % missing. Count missing: 863
title has 0.07 % missing. Count missing: 3
Keywords has 8.94 % missing. Count missing: 393
cast has 0.3 % missing. Count missing: 13
crew has 0.5 % missing. Count missing: 22
